In [1]:
import ee 
import geemap
geemap.ee_initialize()

# Exporting images

In [2]:
Map = geemap.Map()

image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318').select(
    ['B5', 'B4', 'B3']
)

vis_params = {'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]}

Map.centerObject(image, 8)
Map.addLayer(image, vis_params, 'Landsat')
Map

Map(center=[37.47164678275328, -122.14450014746849], controls=(WidgetControl(options=['position', 'transparent…

In [3]:
region = ee.Geometry.BBox(-122.5955, 37.5339, -122.0982, 37.8252)
fc = ee.FeatureCollection(region)
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
Map.addLayer(fc.style(**style), {}, 'ROI')
Map

Map(bottom=25699.0, center=[37.47921744485059, -121.27807617187501], controls=(WidgetControl(options=['positio…

# To local Drive

not allow to direct download the whole Landsat images (file size too big)

In [4]:
geemap.ee_export_image(image, filename= 'landsat.tif', scale=30, region=region)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\zyang\spatial_data_management\notes\landsat.tif


# Projection

In [5]:

projection = image.select(0).projection().getInfo()
projection

{'type': 'Projection',
 'crs': 'EPSG:32610',
 'transform': [30, 0, 460785, 0, -30, 4264215]}

In [6]:
crs = projection['crs']
crs_transform = projection['transform']

In [7]:
geemap.ee_export_image(
    image,
    filename="landsat_crs.tif",
    crs=crs,
    crs_transform=crs_transform,
    region=region,
)
     

Generating URL ...
Please wait ...
Data downloaded to c:\Users\zyang\spatial_data_management\notes\landsat_crs.tif
